In [58]:
import pandas as pd
import numpy as np
import fuzzywuzzy as fuzz
import difflib

### Download Licenses File from Website

In [3]:
data = pd.read_csv('https://www.abc.ca.gov/datport/ABC_Data_Export_CSV.zip')

In [4]:
data['Premise Street Address 2'] = data['Premise Street Address 2'].replace(np.nan, "", regex=True)

In [5]:
data.head()

,License Type,File Number,License or Application,Type Status,Type Original Issue Dates,Expiration Dates,Fee Codes,Duplicate Counts,Master Indicator,Term in # of Months,...,Premise State,Premise Zip,DBA Name,Mail Street Address 1,Mail Street Address 2,Mail City,Mail State,Mail Zip,Premise County,Premise Census Tract #
0,20,100196,LIC,SUREND,21-JAN-1981,30-JUN-2017,P40,NaN,Y,12,...,CA,92201,NaN,NaN,NaN,NaN,NaN,NaN,RIVERSIDE,455.02
1,20,100275,LIC,ACTIVE,18-AUG-1981,31-MAY-2018,P40,NaN,Y,12,...,CA,95822,MILLERS AM PM,NaN,NaN,NaN,NaN,NaN,SACRAMENTO,38.00
2,41,100297,LIC,ACTIVE,09-JUN-1981,31-DEC-2017,P0,NaN,Y,12,...,CA,95448,ROUND TABLE PIZZA,575 W COLLEGE AVE,STE 201,SANTA ROSA,CA,95401,SONOMA,1539.01
3,41,100298,LIC,ACTIVE,05-OCT-1981,31-DEC-2017,P0,NaN,Y,12,...,CA,95453,ROUND TABLE PIZZA,575 W COLLEGE AVE,STE 201,SANTA ROSA,CA,95401,LAKE,4.00
4,9,100306,LIC,ACTIVE,19-MAY-1981,30-JUN-2017,P0,1.0,N,12,...,CA,93514,GILLESPIE DISTRIBUTING COMPANY INC,NaN,NaN,NaN,NaN,NaN,INYO,4.00


In [6]:
data.shape

(115369, 26)

In [7]:
data.columns

Index(['License Type', 'File Number', 'License or Application', 'Type Status',
       'Type Original Issue Dates', 'Expiration Dates', 'Fee Codes',
       'Duplicate Counts', 'Master Indicator', 'Term in # of Months',
       'Geo Code', 'District / Office Code', 'Primary Name',
       'Premise Street Address 1', 'Premise Street Address 2', 'Premise City',
       'Premise State', 'Premise Zip', 'DBA Name', 'Mail Street Address 1',
       'Mail Street Address 2', 'Mail City', 'Mail State', 'Mail Zip',
       'Premise County', 'Premise Census Tract #'],
      dtype='object')

In [170]:
IQstoredata = pd.read_csv('IQCalifornia.csv', dtype = str)
IQstoredata.head()

,RTL_STORE_CD,RTL_TDLINX_STORE_DSC,RTL_ADDR_01_DSC,RTL_CITY_DSC,RTL_STATE_CD,RTL_FIPS_COUNTY_DSC,RTL_POSTAL_CD,RTL_LATITUDE,RTL_LONGITUDE
0,205500986,VINO RESTAURANT,2387 N MAIN ST,WALNUT CREEK,CA,CONTRA COSTA,94596,37.9132,-122.0658
1,105255028,TRACY BILLARDS,130 W 11TH ST STE G,TRACY,CA,SAN JOAQUIN,95376,37.7396,-121.4277
2,105513866,MARISCOS LINDA RESTAURANT,5527 PACIFIC BLVD,HUNTINGTON PARK,CA,LOS ANGELES,90255,33.9923,-118.2254
3,105522121,TAQUERIA RANCHO GRANDE,3330 E FLORIDA AVE STE B,HEMET,CA,RIVERSIDE,92544,33.7479,-116.9365
4,103709760,SALINAS DISCOUNT,648 E ALISAL ST,SALINAS,CA,MONTEREY,93905,36.6734,-121.6336


In [171]:
subsetdata = data.loc[:,['License Type','File Number','Primary Name','Premise Street Address 1', 'Premise Street Address 2', 'Premise City','Premise State', 'Premise Zip']]

In [172]:
subsetdata.head()

,License Type,File Number,Primary Name,Premise Street Address 1,Premise Street Address 2,Premise City,Premise State,Premise Zip
0,20,100196,MEZA ALFREDO T,82675 INDIO BLVD,,INDIO,CA,92201
1,20,100275,D MILLER INC,1391 FLORIN RD,,SACRAMENTO,CA,95822
2,41,100297,WYVERN RESTAURANTS INC,1351 HEALDSBURG AVE,,HEALDSBURG,CA,95448
3,41,100298,WYVERN RESTAURANTS INC,821 11TH ST,,LAKEPORT,CA,95453
4,9,100306,GILLESPIE DISTRIBUTING COMPANY INC,486 E LINE ST,,BISHOP,CA,93514


In [173]:
subsetdata['Premise Street Address 2'].describe()

count     115369
unique      4481
top             
freq       93302
Name: Premise Street Address 2, dtype: object

In [174]:
### Create a Street Column in both tables
subsetdata['Full_Address'] = subsetdata['Premise Street Address 1']+ " " +subsetdata['Premise Street Address 2']+ " "+subsetdata['Premise City'] + " " +subsetdata['Premise State'] + " " +subsetdata['Premise Zip']

In [175]:
### Create a Street Column in subset table
subsetdata.loc[subsetdata['Premise Street Address 2']=="",'Full_Address'] = subsetdata['Premise Street Address 1']+ " " +subsetdata['Premise City'] + " " +subsetdata['Premise State'] + " " +subsetdata['Premise Zip']
subsetdata.loc[subsetdata['Premise Street Address 2']!="",'Full_Address'] = subsetdata['Premise Street Address 1']+ " " +subsetdata['Premise Street Address 2']+ " "+subsetdata['Premise City'] + " " +subsetdata['Premise State'] + " " +subsetdata['Premise Zip']

In [176]:
subsetdata = subsetdata.dropna(axis = 0, how = 'any')

In [177]:
subsetdata['Full_Address'].head()

0            82675 INDIO BLVD INDIO CA 92201
1         1391 FLORIN RD SACRAMENTO CA 95822
2    1351 HEALDSBURG AVE HEALDSBURG CA 95448
3              821 11TH ST LAKEPORT CA 95453
4              486 E LINE ST BISHOP CA 93514
Name: Full_Address, dtype: object

In [178]:
IQstoredata['Full_Address'] = IQstoredata['RTL_ADDR_01_DSC']+" "+IQstoredata['RTL_CITY_DSC'] + " " +IQstoredata['RTL_STATE_CD'] + " " +IQstoredata['RTL_POSTAL_CD']

In [179]:
#create Empty Data Frames
matched_df = pd.DataFrame()

#created unmatched df
unmatched_df = IQstoredata

In [180]:
merge_df = pd.merge(unmatched_df, subsetdata, on = 'Full_Address', how = 'left')
merge_df

,RTL_STORE_CD,RTL_TDLINX_STORE_DSC,RTL_ADDR_01_DSC,RTL_CITY_DSC,RTL_STATE_CD,RTL_FIPS_COUNTY_DSC,RTL_POSTAL_CD,RTL_LATITUDE,RTL_LONGITUDE,Full_Address,License Type,File Number,Primary Name,Premise Street Address 1,Premise Street Address 2,Premise City,Premise State,Premise Zip
0,205500986,VINO RESTAURANT,2387 N MAIN ST,WALNUT CREEK,CA,CONTRA COSTA,94596,37.9132,-122.0658,2387 N MAIN ST WALNUT CREEK CA 94596,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,105255028,TRACY BILLARDS,130 W 11TH ST STE G,TRACY,CA,SAN JOAQUIN,95376,37.7396,-121.4277,130 W 11TH ST STE G TRACY CA 95376,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,105513866,MARISCOS LINDA RESTAURANT,5527 PACIFIC BLVD,HUNTINGTON PARK,CA,LOS ANGELES,90255,33.9923,-118.2254,5527 PACIFIC BLVD HUNTINGTON PARK CA 90255,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,105522121,TAQUERIA RANCHO GRANDE,3330 E FLORIDA AVE STE B,HEMET,CA,RIVERSIDE,92544,33.7479,-116.9365,3330 E FLORIDA AVE STE B HEMET CA 92544,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,103709760,SALINAS DISCOUNT,648 E ALISAL ST,SALINAS,CA,MONTEREY,93905,36.6734,-121.6336,648 E ALISAL ST SALINAS CA 93905,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,205261772,ADOLFO GRILL,67 E DAILY DR,CAMARILLO,CA,VENTURA,93010,34.222,-119.0689,67 E DAILY DR CAMARILLO CA 93010,47.0,550236.0,ADOLFO GRILL INC,67 E DAILY DR,,CAMARILLO,CA,93010
6,102019407,LAKE GREGORY TRADING,23987 LAKE DR,CRESTLINE,CA,SAN BRNRDINO,92325,34.2435,-117.2786,23987 LAKE DR CRESTLINE CA 92325,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,101781692,HAMPTON INN,212 ALEXA CT,PASO ROBLES,CA,SAN L OBISPO,93446,35.5885,-120.697,212 ALEXA CT PASO ROBLES CA 93446,20.0,395154.0,PACIFIC COAST HOTEL PROPERTIES I LLC,212 ALEXA CT,,PASO ROBLES,CA,93446
8,101781692,HAMPTON INN,212 ALEXA CT,PASO ROBLES,CA,SAN L OBISPO,93446,35.5885,-120.697,212 ALEXA CT PASO ROBLES CA 93446,70.0,395154.0,PACIFIC COAST HOTEL PROPERTIES I LLC,212 ALEXA CT,,PASO ROBLES,CA,93446
9,105519058,ROUND TABLE PIZZA,4302 MOORPARK AVE,SAN JOSE,CA,SANTA CLARA,95129,37.3159,-121.9779,4302 MOORPARK AVE SAN JOSE CA 95129,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [181]:
merge_result = merge_df.loc[merge_df['License Type'].isnull() == False]
merge_result.head(100)

,RTL_STORE_CD,RTL_TDLINX_STORE_DSC,RTL_ADDR_01_DSC,RTL_CITY_DSC,RTL_STATE_CD,RTL_FIPS_COUNTY_DSC,RTL_POSTAL_CD,RTL_LATITUDE,RTL_LONGITUDE,Full_Address,License Type,File Number,Primary Name,Premise Street Address 1,Premise Street Address 2,Premise City,Premise State,Premise Zip
5,205261772,ADOLFO GRILL,67 E DAILY DR,CAMARILLO,CA,VENTURA,93010,34.222,-119.0689,67 E DAILY DR CAMARILLO CA 93010,47.0,550236.0,ADOLFO GRILL INC,67 E DAILY DR,,CAMARILLO,CA,93010
7,101781692,HAMPTON INN,212 ALEXA CT,PASO ROBLES,CA,SAN L OBISPO,93446,35.5885,-120.697,212 ALEXA CT PASO ROBLES CA 93446,20.0,395154.0,PACIFIC COAST HOTEL PROPERTIES I LLC,212 ALEXA CT,,PASO ROBLES,CA,93446
8,101781692,HAMPTON INN,212 ALEXA CT,PASO ROBLES,CA,SAN L OBISPO,93446,35.5885,-120.697,212 ALEXA CT PASO ROBLES CA 93446,70.0,395154.0,PACIFIC COAST HOTEL PROPERTIES I LLC,212 ALEXA CT,,PASO ROBLES,CA,93446
15,102567795,WINCO FOODS,19047 BEAR VALLEY RD,APPLE VALLEY,CA,SAN BRNRDINO,92308,34.4708,-117.248,19047 BEAR VALLEY RD APPLE VALLEY CA 92308,21.0,459122.0,WINCO FOODS LLC,19047 BEAR VALLEY RD,,APPLE VALLEY,CA,92308
16,102567795,WINCO FOODS,19047 BEAR VALLEY RD,APPLE VALLEY,CA,SAN BRNRDINO,92308,34.4708,-117.248,19047 BEAR VALLEY RD APPLE VALLEY CA 92308,86.0,459122.0,WINCO FOODS LLC,19047 BEAR VALLEY RD,,APPLE VALLEY,CA,92308
29,105245135,SWISS PARK,5911 MOWRY AVE,NEWARK,CA,ALAMEDA,94560,37.5263,-122.0069,5911 MOWRY AVE NEWARK CA 94560,47.0,35467.0,NEWARK SWISS PARK INC,5911 MOWRY AVE,,NEWARK,CA,94560
30,105245135,SWISS PARK,5911 MOWRY AVE,NEWARK,CA,ALAMEDA,94560,37.5263,-122.0069,5911 MOWRY AVE NEWARK CA 94560,47.0,35467.0,NEWARK SWISS PARK INC,5911 MOWRY AVE,,NEWARK,CA,94560
34,107488234,SAFEWAY STORE,709 HICKEY BLVD,PACIFICA,CA,SAN MATEO,94044,37.66,-122.4779,709 HICKEY BLVD PACIFICA CA 94044,21.0,449781.0,LUCKY STORES INC,709 HICKEY BLVD,,PACIFICA,CA,94044
35,107488234,SAFEWAY STORE,709 HICKEY BLVD,PACIFICA,CA,SAN MATEO,94044,37.66,-122.4779,709 HICKEY BLVD PACIFICA CA 94044,21.0,549278.0,SAFEWAY INC,709 HICKEY BLVD,,PACIFICA,CA,94044
42,101961071,CAFE SOLAZ,60750 TRILOGY PKWY,LA QUINTA,CA,RIVERSIDE,92253,33.6075,-116.242,60750 TRILOGY PKWY LA QUINTA CA 92253,47.0,418566.0,TRILOGY AT LA QUINTA MAINTENANCE ASSOCIATION,60750 TRILOGY PKWY,,LA QUINTA,CA,92253


In [182]:
matched_df = merge_result
matched_df.head()

,RTL_STORE_CD,RTL_TDLINX_STORE_DSC,RTL_ADDR_01_DSC,RTL_CITY_DSC,RTL_STATE_CD,RTL_FIPS_COUNTY_DSC,RTL_POSTAL_CD,RTL_LATITUDE,RTL_LONGITUDE,Full_Address,License Type,File Number,Primary Name,Premise Street Address 1,Premise Street Address 2,Premise City,Premise State,Premise Zip
5,205261772,ADOLFO GRILL,67 E DAILY DR,CAMARILLO,CA,VENTURA,93010,34.222,-119.0689,67 E DAILY DR CAMARILLO CA 93010,47.0,550236.0,ADOLFO GRILL INC,67 E DAILY DR,,CAMARILLO,CA,93010
7,101781692,HAMPTON INN,212 ALEXA CT,PASO ROBLES,CA,SAN L OBISPO,93446,35.5885,-120.697,212 ALEXA CT PASO ROBLES CA 93446,20.0,395154.0,PACIFIC COAST HOTEL PROPERTIES I LLC,212 ALEXA CT,,PASO ROBLES,CA,93446
8,101781692,HAMPTON INN,212 ALEXA CT,PASO ROBLES,CA,SAN L OBISPO,93446,35.5885,-120.697,212 ALEXA CT PASO ROBLES CA 93446,70.0,395154.0,PACIFIC COAST HOTEL PROPERTIES I LLC,212 ALEXA CT,,PASO ROBLES,CA,93446
15,102567795,WINCO FOODS,19047 BEAR VALLEY RD,APPLE VALLEY,CA,SAN BRNRDINO,92308,34.4708,-117.248,19047 BEAR VALLEY RD APPLE VALLEY CA 92308,21.0,459122.0,WINCO FOODS LLC,19047 BEAR VALLEY RD,,APPLE VALLEY,CA,92308
16,102567795,WINCO FOODS,19047 BEAR VALLEY RD,APPLE VALLEY,CA,SAN BRNRDINO,92308,34.4708,-117.248,19047 BEAR VALLEY RD APPLE VALLEY CA 92308,86.0,459122.0,WINCO FOODS LLC,19047 BEAR VALLEY RD,,APPLE VALLEY,CA,92308


In [183]:
unmatched_df = unmatched_df.loc[-unmatched_df['RTL_STORE_CD'].isin(matched_df['RTL_STORE_CD']), :]
unmatched_df.head()

,RTL_STORE_CD,RTL_TDLINX_STORE_DSC,RTL_ADDR_01_DSC,RTL_CITY_DSC,RTL_STATE_CD,RTL_FIPS_COUNTY_DSC,RTL_POSTAL_CD,RTL_LATITUDE,RTL_LONGITUDE,Full_Address
0,205500986,VINO RESTAURANT,2387 N MAIN ST,WALNUT CREEK,CA,CONTRA COSTA,94596,37.9132,-122.0658,2387 N MAIN ST WALNUT CREEK CA 94596
1,105255028,TRACY BILLARDS,130 W 11TH ST STE G,TRACY,CA,SAN JOAQUIN,95376,37.7396,-121.4277,130 W 11TH ST STE G TRACY CA 95376
2,105513866,MARISCOS LINDA RESTAURANT,5527 PACIFIC BLVD,HUNTINGTON PARK,CA,LOS ANGELES,90255,33.9923,-118.2254,5527 PACIFIC BLVD HUNTINGTON PARK CA 90255
3,105522121,TAQUERIA RANCHO GRANDE,3330 E FLORIDA AVE STE B,HEMET,CA,RIVERSIDE,92544,33.7479,-116.9365,3330 E FLORIDA AVE STE B HEMET CA 92544
4,103709760,SALINAS DISCOUNT,648 E ALISAL ST,SALINAS,CA,MONTEREY,93905,36.6734,-121.6336,648 E ALISAL ST SALINAS CA 93905


In [184]:
subsetdata = subsetdata.loc[-subsetdata['File Number'].isin(matched_df['File Number']), :]
subsetdata.head()

,License Type,File Number,Primary Name,Premise Street Address 1,Premise Street Address 2,Premise City,Premise State,Premise Zip,Full_Address
1,20,100275,D MILLER INC,1391 FLORIN RD,,SACRAMENTO,CA,95822,1391 FLORIN RD SACRAMENTO CA 95822
18,20,101076,MORITA ALBERT K,3479 MOTOR AVE,,LOS ANGELES,CA,90034,3479 MOTOR AVE LOS ANGELES CA 90034
20,21,101277,HUGHES MARKETS INC,583 N VENTU PARK RD,,THOUSAND OAKS,CA,91320,583 N VENTU PARK RD THOUSAND OAKS CA 91320
21,41,101355,GAUL GEORGE F JR,7400 FLORENCE AVE,,DOWNEY,CA,90240-3609,7400 FLORENCE AVE DOWNEY CA 90240-3609
26,47,101602,RAINBOW BALLROOM INC,1725 BROADWAY ST,,FRESNO,CA,93721-1620,1725 BROADWAY ST FRESNO CA 93721-1620


In [191]:
difflib.get_close_matches(unmatched_df.loc[0, 'Full_Address'], list(subsetdata['Full_Address']), n =1, cutoff = 0.85)

['2355 N MAIN ST WALNUT CREEK CA 94596-3547']

In [192]:
unmatched_df.loc[0, 'Full_Address']

'2387 N MAIN ST WALNUT CREEK CA 94596'

In [193]:
subsetdata.loc[subsetdata['Full_Address']=='2355 N MAIN ST WALNUT CREEK CA 94596-3547']

,License Type,File Number,Primary Name,Premise Street Address 1,Premise Street Address 2,Premise City,Premise State,Premise Zip,Full_Address
68212,47,526976,WINDSOR CAPITAL GROUP INC,2355 N MAIN ST,,WALNUT CREEK,CA,94596-3547,2355 N MAIN ST WALNUT CREEK CA 94596-3547
68213,68,526976,WINDSOR CAPITAL GROUP INC,2355 N MAIN ST,,WALNUT CREEK,CA,94596-3547,2355 N MAIN ST WALNUT CREEK CA 94596-3547


In [62]:
keyword_df = unmatched_df
for index in range(len(unmatched_df)):
    word = str(unmatched_df.loc[index, 'Full_Address'])
    result = difflib.get_close_matches(word, list(subsetdata['Full_Address']))
    #keyword_df.loc[index, 'result'] = result


TypeError: object of type 'float' has no len()

In [69]:
unmatched_df.loc[0, 'Full_Address']

'2387 N MAIN ST WALNUT CREEK CA 94596'

In [169]:
tempList = list(subsetdata['Full_Address'])[118]
tempList

nan

In [168]:
difflib.get_close_matches('2387 N MAIN ST WALNUT CREEK CA 94596', tempList)

TypeError: 'float' object is not iterable

In [77]:

fruit = ['ape', 'apple', 'peachyyyy', 'puppy', 'afde']
difflib.get_close_matches('appel', fruit, n = 1, cutoff = 0.8)


5

In [42]:
unmatched_df.head()

,RTL_STORE_CD,RTL_TDLINX_STORE_DSC,RTL_ADDR_01_DSC,RTL_CITY_DSC,RTL_STATE_CD,RTL_FIPS_COUNTY_DSC,RTL_POSTAL_CD,RTL_LATITUDE,RTL_LONGITUDE,Full_Address
0,205500986,VINO RESTAURANT,2387 N MAIN ST,WALNUT CREEK,CA,CONTRA COSTA,94596,37.9132,-122.0658,2387 N MAIN ST WALNUT CREEK CA 94596
1,105255028,TRACY BILLARDS,130 W 11TH ST STE G,TRACY,CA,SAN JOAQUIN,95376,37.7396,-121.4277,130 W 11TH ST STE G TRACY CA 95376
2,105513866,MARISCOS LINDA RESTAURANT,5527 PACIFIC BLVD,HUNTINGTON PARK,CA,LOS ANGELES,90255,33.9923,-118.2254,5527 PACIFIC BLVD HUNTINGTON PARK CA 90255
3,105522121,TAQUERIA RANCHO GRANDE,3330 E FLORIDA AVE STE B,HEMET,CA,RIVERSIDE,92544,33.7479,-116.9365,3330 E FLORIDA AVE STE B HEMET CA 92544
4,103709760,SALINAS DISCOUNT,648 E ALISAL ST,SALINAS,CA,MONTEREY,93905,36.6734,-121.6336,648 E ALISAL ST SALINAS CA 93905


In [ ]:
for index_store in range(1):
    for index_gov in range(len())

In [47]:
fuzz.ratio()